### Импорты

In [68]:
import numpy as np
import time
import memory_profiler
import functools
import math
import pandas as pd
import plotly.graph_objects as go

### Задание 1. Сравнительный анализ производительности NumPy и чистого Python

In [69]:
def generate_test_datasets() -> dict[str, np.array]:
    return {
        'small': np.random.random(1000),
        'medium': np.random.random(10000),
        'large': np.random.random(100000),
        'xlarge': np.random.random(1000000)
        }

In [70]:
def benchmark(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        _ = func(*args, **kwargs)
        end_time = time.time()
        #   аргумент retval=True позволит вренуть результат вычислений
        #       но это может исказить точное значение времени выполнения исходной функции
        memory_usage = memory_profiler.memory_usage((func, args, kwargs), max_usage=True)

        return {
            'time': end_time - start_time,
            'max_memory': memory_usage,
        }
    return wrapper

In [71]:
@benchmark
def py_square(data):
    return [x ** 2 for x in data]

@benchmark
def np_square(data):
    return np.square(data)

@benchmark
def py_sin(data):
    return [math.sin(x) for x in data]

@benchmark
def np_sin(data):
    return np.sin(data)

@benchmark
def py_sum(data):
    return sum(data)

@benchmark
def np_sum(data):
    return np.sum(data)

@benchmark
def py_max(data):
    return max(data)

@benchmark
def np_max(data):
    return np.max(data)

In [ ]:
#   при первом запуске функции происходит компиляция,
#       что может исказить результаты замеров времени выполнения
#       (при первом запуске numpy уступает по скорости чистому python)

def benchmark_operations():
    datasets = generate_test_datasets()
    operations = [
        py_square,
        np_square,
        py_sin,
        np_sin,
        py_sum,
        np_sum,
        py_max,
        np_max
    ]
    all_results = {}
    for name, data in datasets.items():
        for operation in operations:
            operation_results = all_results.setdefault(operation.__name__, {})
            operation_results[name] = operation(data)

    return all_results

In [73]:
benchmark_reslults = benchmark_operations()

In [74]:
benchmark_reslults

{'py_square': {'small': {'time': 0.0008692741394042969,
   'max_memory': 237.16796875},
  'medium': {'time': 0.0013880729675292969, 'max_memory': 238.01953125},
  'large': {'time': 0.013671398162841797, 'max_memory': 245.5078125},
  'xlarge': {'time': 0.1547391414642334, 'max_memory': 351.20703125}},
 'np_square': {'small': {'time': 2.1696090698242188e-05,
   'max_memory': 237.16796875},
  'medium': {'time': 3.981590270996094e-05, 'max_memory': 234.6796875},
  'large': {'time': 0.0001316070556640625, 'max_memory': 238.5703125},
  'xlarge': {'time': 0.0017390251159667969, 'max_memory': 266.24609375}},
 'py_sin': {'small': {'time': 0.0001704692840576172,
   'max_memory': 237.16796875},
  'medium': {'time': 0.001123189926147461, 'max_memory': 234.6796875},
  'large': {'time': 0.012008428573608398, 'max_memory': 245.5078125},
  'xlarge': {'time': 0.12671208381652832, 'max_memory': 351.90234375}},
 'np_sin': {'small': {'time': 2.5510787963867188e-05,
   'max_memory': 237.16796875},
  'mediu

In [75]:
rows = []
for func, sizes in benchmark_reslults.items():
    for size, metrics in sizes.items():
        rows.append({
            'function': func,
            'size': size,
            'time': metrics['time'],
            'memory': metrics['max_memory']
        })

df = pd.DataFrame(rows)

In [76]:
df.head()

,function,size,time,memory
0,py_square,small,0.000869,237.167969
1,py_square,medium,0.001388,238.019531
2,py_square,large,0.013671,245.507812
3,py_square,xlarge,0.154739,351.207031
4,np_square,small,0.000022,237.167969


In [77]:
# Упорядочиваем размеры
size_order = ['small', 'medium', 'large', 'xlarge']
df['size'] = pd.Categorical(df['size'], categories=size_order, ordered=True)
df = df.sort_values(['function', 'size'])

# Создаем график для времени выполнения
fig_time = go.Figure()

# Цвета для функций
colors = {
    'py_square': '#FF6B6B',
    'np_square': '#4ECDC4',
    'py_sin': '#FFD166',
    'np_sin': '#06D6A0',
    'py_sum': '#118AB2',
    'np_sum': '#EF476F',
    'py_max': '#073B4C',
    'np_max': '#7209B7'
}

# Добавляем trace для каждой функции
for func in df['function'].unique():
    func_data = df[df['function'] == func]
    fig_time.add_trace(go.Bar(
        name=func,
        x=func_data['size'],
        y=func_data['time'],
        marker_color=colors.get(func, 'gray'),
        text=[f'{t:.6f}' for t in func_data['time']],
        textposition='auto',
    ))

fig_time.update_layout(
    title='Время выполнения функций по размерам данных',
    xaxis_title='Размер данных',
    yaxis_title='Время (сек)',
    barmode='group',
    yaxis_type='log',  # Логарифмическая шкала для наглядности
    height=600,
    showlegend=True
)

fig_time.show()

In [78]:
size_order = ['small', 'medium', 'large', 'xlarge']
df['size'] = pd.Categorical(df['size'], categories=size_order, ordered=True)
df = df.sort_values(['function', 'size'])

# Создаем график для времени выполнения
fig_time = go.Figure()

# Цвета для функций
colors = {
    'py_square': '#FF6B6B',
    'np_square': '#4ECDC4',
    'py_sin': '#FFD166',
    'np_sin': '#06D6A0',
    'py_sum': '#118AB2',
    'np_sum': '#EF476F',
    'py_max': '#073B4C',
    'np_max': '#7209B7'
}

# Добавляем trace для каждой функции
for func in df['function'].unique():
    func_data = df[df['function'] == func]
    fig_time.add_trace(go.Bar(
        name=func,
        x=func_data['size'],
        y=func_data['memory'],
        marker_color=colors.get(func, 'gray'),
        text=[f'{t:.6f}' for t in func_data['memory']],
        textposition='auto',
    ))

fig_time.update_layout(
    title='Затраты памяти на выполнения функций по размерам данных',
    xaxis_title='Размер данных',
    yaxis_title='Память',
    barmode='group',
    yaxis_type='linear',
    height=600,
    showlegend=True
)

fig_time.show()

In [114]:
# Строим сводную таблицу времени выполнения
pivot_time = df.pivot_table(index='function', columns='size', values='time', observed=True)

# Вычисляем ускорение (speedup) для каждой функции и размера
speedup_data = []
sizes = list(pivot_time.columns)
functions = list(set(map(lambda x: x.split("_")[1], pivot_time.index)))

for func in functions:
    row = []
    for size in sizes:
        py_time = pivot_time.loc[f'py_{func}', size]
        np_time = pivot_time.loc[f'np_{func}', size]
        speedup = py_time / np_time if np_time > 0 else 0
        row.append(speedup)
    speedup_data.append(row)

# Создаем DataFrame для тепловой карты
speedup_df = pd.DataFrame(speedup_data, index=functions, columns=sizes)

fig = go.Figure(data=go.Heatmap(
    z=speedup_df.values,
    x=speedup_df.columns,
    y=speedup_df.index,
    text=np.round(speedup_df.values, 2),
    texttemplate='%{text}',
    textfont={"size": 16},
    colorscale = 'ylgn',
    hoverongaps=False,
    hovertemplate='<b>Функция</b>: %{y}<br>' +
                  '<b>Размер</b>: %{x}<br>' +
                  '<b>Ускорение</b>: %{z:.2f}x<br>' +
                  '<extra></extra>',))
# Настраиваем макет
fig.update_layout(
    title={
        'text': 'Тепловая карта относительного ускорения NumPy над чистым Python',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Размер данных",
    yaxis_title="Функция",
)

fig.show()

### 2. Анализ и визуализация данных с pandas и seaborn